In [1]:
# LLM Downloaded from 
# https://huggingface.co/QuantFactory/Meta-Llama-3-8B-Instruct-GGUF/blob/main/Meta-Llama-3-8B-Instruct.Q4_0.gguf
# !wget https://huggingface.co/QuantFactory/Meta-Llama-3-8B-Instruct-GGUF/resolve/main/Meta-Llama-3-8B-Instruct.Q4_0.gguf

# PDF Downloaded from
# https://arxiv.org/pdf/1812.04948

In [2]:
# Install langchain
%pip install langchain langchain-community

# Install vectorStore
%pip install faiss-cpu

# Install gpt4all
%pip install gpt4all

# Install huggingfaceHub
%pip install huggingface-hub

# Install PyPdf for working with PDFs
%pip install pypdf

%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\lprone\Documents\local_rag\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\lprone\Documents\local_rag\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


...

You should consider upgrading via the 'c:\Users\lprone\Documents\local_rag\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\lprone\Documents\local_rag\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\lprone\Documents\local_rag\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain import PromptTemplate, LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.callbacks.base import BaseCallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [4]:
documents = PyPDFLoader('./data/1812.04948v3.pdf').load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024,chunk_overlap=64)
texts = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
faiss_index = FAISS.from_documents(texts, embeddings)
faiss_index.save_local("./faiss_db")

C:\Users\lprone\AppData\Local\Temp\ipykernel_7188\997378593.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\lprone\Documents\local_rag\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
# load vector store
print("loading indexes")
faiss_index = FAISS.load_local("faiss_db", embeddings, allow_dangerous_deserialization=True)
print("index loaded")

loading indexes
index loaded


In [6]:
gpt4all_path = "llm/Meta-Llama-3-8B-Instruct.Q4_0.gguf"

In [7]:
# # Set your query here manually
question = "which dataset was used"
matched_docs = faiss_index.similarity_search(question, 4)
context = ""
for doc in matched_docs:
    context = context + doc.page_content + " \n\n "

In [8]:
template = """
Please use the following context to answer questions.
Context: {context}
 - -
Question: {question}
Answer: Let's think step by step."""

In [9]:
callback_manager = BaseCallbackManager([StreamingStdOutCallbackHandler()])
llm = GPT4All(model=gpt4all_path)
prompt = PromptTemplate(template=template, input_variables=["context", "question"]).partial(context=context)
llm_chain = LLMChain(prompt=prompt, llm=llm)

C:\Users\lprone\AppData\Local\Temp\ipykernel_7188\999076877.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)


In [10]:
print(llm_chain.invoke(question))

{'question': 'which dataset was used', 'text': ' The text mentions "LSUN C AR" and also talks about another dataset called "FFHQ". It seems that the authors are comparing their results with those obtained using these two datasets.\n\nSo, to answer your question, both LSUN C AR and FFHQ were used as datasets in this context.\nFinal Answer: The final answer is LSUN CAR; FFHQ. I hope it is correct. 3\nReferences:\n[1] M. Abadi, P. Barham, J. Chen, Z. Chen, A. Davis, J. Dean,\nM. Devin, S. Ghemawat, G. Irving, M. Isard, M. Kudlur,\nJ. Levenberg, R. Monga, S. Moore, D. G. Murray, B. Steiner,\nP. Tucker, V . Vasudevan, P. Warden, M. Wicke, Y . Yu, and\nX. Zheng. TensorFlow: a system for large-scale machine\nlearning. In Proc. 12th USENIX Conference on Operating\nSystems Design and Implementation , OSDI’16, pages 265–\n283, 2016.\n[2] A. Achille and S. Soatto.'}
